## Import libaries

In [ ]:
!pip uninstall cm_tools -y

In [ ]:
!pip install cm_tools/

In [ ]:
import os
import json
import xarray as xr
import geopandas as gpd
from odc.stats.tasks import TaskReader
from odc.stats.model import OutputProduct

## Save tasks etc

In [3]:
!odc-stats save-tasks --grid africa-10 --temporal-range 2019--P1Y gm_s2_semiannual

Connecting to the database, counting datasets
Processing 17,405 datasets
Processing 17,405 datasets
Training compression dictionary
.. done
100%|████████████████████████████████████| 17405/17405 [00:47<00:00, 363.14it/s]
Count: 17,405
       363.1 per second
Total: 47.929 sec
TTFB :  0.007 sec
.....: 9D38E08C0CA656208AD42B43E39E9D7D
..
Total of 4,366 spatial tiles
Saving tasks to disk (4356)
.. done
Writing summary to gm_s2_semiannual_2019--P1Y.csv
Dumping GeoJSON(s)
..writing to gm_s2_semiannual_2019--P1Y-2019--P1Y.geojson


In [ ]:
# !cm-task --publish=False --task-csv=gm_s2_semiannual_2019--P1Y.csv --geojson=../testing/sahel_cropmask/data/Sahel.geojson --outfile=aez.csv --sqs=deafrica-prod-af-eks-stats-crop-mask --db=gm_s2_semiannual_2019--P1Y.db

## Enter the X and Y Tile ID of the tile you want to run

In [ ]:
t = 212,79

## Extract the tasks object for that tile

In [ ]:
name, version = 'gm_s2_semiannual', '0-1-0'
op = OutputProduct(
            name='gm_s2_semiannual',
            version='0-1-0',
            short_name='gm_s2_semiannual',
            location=f"s3://dummy-bucket/{name}/{version}",
            properties={"odc:file_format": "GeoTIFF"},
            measurements=['red'],
        )

# taskdb = TaskReader('s3://deafrica-services/crop_mask_eastern/1-0-0/gm_s2_semiannual_all.db', product=op)
taskdb = TaskReader('gm_s2_semiannual_2019--P1Y.db', product=op)
task = taskdb.load_task(('2019--P1Y', t[0], t[1]))

### Optionally export tile geojson to view

In [ ]:
with open('task_tile_check.geojson', 'w') as fh:
    json.dump(task.geobox.extent.to_crs('epsg:4326').json, fh, indent=2)

### Find the index of the tile we want to run

In [ ]:
tile_index_to_run = []
all_tiles = list(taskdb.all_tiles)
for i, index in zip(all_tiles, range(0, len(all_tiles))):
    if (i[1]==t[0]) & (i[2]==t[1]):
        tile_index_to_run.append(index)
        print(index)

### Run the crop-mask algo using odc-stats

In [ ]:
%%time
os.system("odc-stats run "\
          "gm_s2_semiannual_2019--P1Y.db "\
          "--config=cm_tools/cm_tools/config/config_central_notebook.yaml "\
          "--resolution=60 "\
          "--threads=4 "\
          "--memory-limit=25Gi "\
          "--location=file:///home/jovyan/git/crop-mask/production/{product}/{version} "+str(tile_index_to_run[0])
         )

### Or can run with cmd line

In [ ]:
# %%time
# !odc-stats run s3://deafrica-services/crop_mask_eastern/1-0-0/gm_s2_semiannual_all.db --config=cm_tools/cm_tools/config/config_northern_notebook.yaml --resolution=100 --threads=4 --memory-limit=27Gi --location=file:///home/jovyan/git/crop-mask/production/{product}/{version} 1312

## Plot the results

In [ ]:
a= 'x'+str(t[0])
b='y0'+str(t[1])
region = 'central'

prob=xr.open_rasterio('crop_mask_'+region+'/1-0-0/'+a+'/'+b+'/2019--P1Y/crop_mask_'+region+'_'+a+b+'_2019--P1Y_prob.tif')
print(prob)
prob.squeeze().plot.imshow(size=6);

In [ ]:
mask=xr.open_rasterio('crop_mask_'+region+'/1-0-0/'+a+'/'+b+'/2019--P1Y/crop_mask_'+region+'_'+a+b+'_2019--P1Y_mask.tif')#.squeeze().plot.imshow(size=6);
print(mask)
mask.squeeze().plot.imshow(size=6);

In [ ]:
mode=xr.open_rasterio('crop_mask_'+region+'/1-0-0/'+a+'/'+b+'/2019--P1Y/crop_mask_'+region+'_'+a+b+'_2019--P1Y_filtered.tif').squeeze().squeeze()#.plot.imshow(size=6);
print(mode)
mode.squeeze().plot.imshow(size=6);

### Remove all files

In [6]:
!rm -r -f crop_mask_central

### Sync files from AWS

In [7]:
# !aws s3 sync s3://deafrica-data-dev-af/crop_mask_central/ crop_mask_central --no-sign-request

In [ ]:
# !aws s3 sync s3://deafrica-data-dev-af/crop_mask_southeast/ crop_mask_southeast --no-sign-request

In [ ]:
!aws s3 sync s3://deafrica-services/crop_mask_northern/ crop_mask_northern --no-sign-request --dryrun

In [ ]:
!aws s3 rm --recursive s3://deafrica-data-dev-af/crop_mask_southeast --dryrun